In [12]:
import functools
from typing import Union, Callable

import clingo
from clingox.program import Program, ProgramObserver

In [13]:
static_rules = """

pos((1..3,1..3)).
adjacent((X1,Y1), (X2,Y2)) :- pos((X1,Y1)), pos((X2,Y2)), |X1 - X2| + |Y1 - Y2| = 1.

1 { observe(object(Type, Id), value(Property, Value), 0) : value(Property, Value) } 1 :- object(Type, Id).

value(at, Pos) :- pos(Pos).
action(move, Pos) :- pos(Pos).
action(wait, ()).

"""

In [14]:
effect_rules = """

observe(O, value(at, Pos2), T+1) :-
  time(T+1), time(T),
  occurs(O, action(move, Pos2), T),
  observe(O, value(at, Pos1), T),
  adjacent(Pos1, Pos2).

observe(O, value(at, Pos), T+1) :-
  time(T+1), time(T),
  occurs(O, action(wait, ()), T),
  observe(O, value(at, Pos), T).

observe(object(cell, Pos), value(collision, ()), T) :-
  observe(O1, value(at, Pos), T),
  observe(O2, value(at, Pos), T),
  O1 != O2.

observe(object(cell, Pos), value(collision, ()), T+1) :-
  O1 != O2,
  observe(O1, value(at, Pos), T),
  occurs(O2, action(move, Pos), T).

:- occurs(O, action(move, Pos2), T),                                 % Action
   time(T+1), time(T),
   observe(O, value(at, Pos1), T), adjacent(Pos1, Pos2),             % State
   observe(object(cell, Pos2), value(collision, ()), T+1).           % Precondition

"""

In [15]:
instance = """

object(robot, worker).
object(robot, charger).

init(object(robot, worker), value(at, (1,1))).
init(object(robot, charger), value(at, (2,2))).

goal(object(robot, worker), value(at, (3,3))).
goal(object(robot, charger), value(at, (2,2))).

occurs(object(robot, worker), action(move, (1,2)), 0).
occurs(object(robot, charger), action(move, (2,1)), 0).

occurs(object(robot, worker), action(move, (2,2)), 1).
occurs(object(robot, charger), action(wait, ()), 1).

occurs(object(robot, worker), action(move, (3,2)), 2).
occurs(object(robot, charger), action(wait, ()), 2).

occurs(object(robot, worker), action(move, (3,3)), 3).
occurs(object(robot, charger), action(move, (2,2)), 3).

occurs(object(robot, worker), action(wait, ()), 4).
occurs(object(robot, charger), action(wait, ()), 4).
"""

In [16]:
plan = """

time(0..k).

:- init(O, V), not observe(O, V, 0).
:- goal(O, V), not observe(O, V, k).

"""

In [17]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0
prg = Program()
obs = ProgramObserver(prg)
ctl.register_observer(obs)

In [18]:
ctl.add('base', [], static_rules)
ctl.add('base', [], effect_rules)
ctl.add('base', [], instance)
ctl.add('plan', ['k'], plan)
ctl.add('base', [], '#show observe/3. #show occurs/3.')

In [19]:
ctl.ground([("base", []), ("plan", [clingo.Number(5)])])

In [20]:
print("Facts:", len(prg.facts), "Rules:", len(prg.rules))
print(prg)

Facts: 10 Rules: 126
occurs(object(robot,charger),action(move,(2,1)),0).
occurs(object(robot,charger),action(move,(2,2)),3).
occurs(object(robot,charger),action(wait,()),1).
occurs(object(robot,charger),action(wait,()),2).
occurs(object(robot,charger),action(wait,()),4).
occurs(object(robot,worker),action(move,(1,2)),0).
occurs(object(robot,worker),action(move,(2,2)),1).
occurs(object(robot,worker),action(move,(3,2)),2).
occurs(object(robot,worker),action(move,(3,3)),3).
occurs(object(robot,worker),action(wait,()),4).
 :- not observe(object(robot,worker),value(at,(3,3)),5).
 :- not observe(object(robot,charger),value(at,(2,2)),5).
 :- not observe(object(robot,worker),value(at,(1,1)),0).
 :- not observe(object(robot,charger),value(at,(2,2)),0).
 :- __x36, not __x118.
 :- __x37, not __x121.
 :- observe(object(cell,(1,2)),value(collision,()),1), observe(object(robot,worker),value(at,(2,2)),0).
 :- observe(object(cell,(1,2)),value(collision,()),1), observe(object(robot,worker),value(at,(1,

In [21]:
def get_time(symbol: clingo.Symbol, default: Union[Callable[[clingo.Symbol], int], int, None] = None):
    if symbol.type is clingo.SymbolType.Function and symbol.name in ('occurs', 'observe'):
        return symbol.arguments[-1].number
    if callable(default):
        return default(symbol)
    return default

In [22]:
models = []
with ctl.solve(yield_=True) as solve_handle:
    for model in solve_handle:
        symbols = sorted(sorted(model.symbols(shown=True)), key=functools.partial(get_time, default=-1))
        print("Answer {}:{}".format(model.number,'{'), '\n'.join(map(str, symbols)), '}', sep='\n')
        models.append(symbols)
    solve_result = solve_handle.get()
    print(solve_result, end='')
    if solve_result.satisfiable:
        print(' {}{}'.format(len(models), '' if solve_result.exhausted else '+'), end='')
    print()


Answer 1:{
observe(object(robot,charger),value(at,(2,2)),0)
observe(object(robot,worker),value(at,(1,1)),0)
occurs(object(robot,charger),action(move,(2,1)),0)
occurs(object(robot,worker),action(move,(1,2)),0)
observe(object(robot,charger),value(at,(2,1)),1)
observe(object(robot,worker),value(at,(1,2)),1)
occurs(object(robot,charger),action(wait,()),1)
occurs(object(robot,worker),action(move,(2,2)),1)
observe(object(robot,charger),value(at,(2,1)),2)
observe(object(robot,worker),value(at,(2,2)),2)
occurs(object(robot,charger),action(wait,()),2)
occurs(object(robot,worker),action(move,(3,2)),2)
observe(object(robot,charger),value(at,(2,1)),3)
observe(object(robot,worker),value(at,(3,2)),3)
occurs(object(robot,charger),action(move,(2,2)),3)
occurs(object(robot,worker),action(move,(3,3)),3)
observe(object(robot,charger),value(at,(2,2)),4)
observe(object(robot,worker),value(at,(3,3)),4)
occurs(object(robot,charger),action(wait,()),4)
occurs(object(robot,worker),action(wait,()),4)
observe(obj